

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_Fr_Sentiment.ipynb)






# **Sentiment Analysis of French texts**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.1

In [ ]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start Spark Session

In [ ]:
spark = sparknlp.start()

## 3. Some sample examples

In [ ]:
text_list = [
"""Jeu et championnat pas assez excitants ? Devez-vous vérifier l'arbitre vidéo maintenant? Je suis horrifié, pensa-t-il, il ne devrait répondre que s'il a fait des erreurs flagrantes. Le football n'est plus amusant.""",             
"""J'ai raté le podcast werder hier mercredi. À quelle vitesse vous vous habituez à quelque chose et vous l'attendez avec impatience. Merci à Plainsman pour les bonnes interviews et la perspicacité dans les coulisses du werderbremen. Passez de bonnes vacances d'hiver !""",
"""Les scènes s'enchaînent de manière saccadée, les dialogues sont théâtraux, le jeu des acteurs ne transcende pas franchement le film. Seule la musique de Vivaldi sauve le tout. Belle déception.""",
"""Je n'aime pas l'arbitre parce qu'il est empoisonné !""",
"""ManCity Guardiola et sa bande, vous êtes des connards. Je viens de perdre une fortune à cause de ta dette envers tes Bavarois là-bas.""",
]

## 4. Define Spark NLP pipeline

In [ ]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = BertSentenceEmbeddings\
    .pretrained('labse', 'xx') \
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

sentimentClassifier = ClassifierDLModel.pretrained("classifierdl_bert_sentiment", "fr") \
  .setInputCols(["document", "sentence_embeddings"]) \
  .setOutputCol("class_")    
     
nlpPipeline = Pipeline(
    stages=[
        document, 
        embeddings,
        sentimentClassifier])

labse download started this may take some time.
Approximate size to download 1.7 GB
[OK!]
classifierdl_bert_sentiment download started this may take some time.
Approximate size to download 22.2 MB
[OK!]


## 5. Run the pipeline

In [ ]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlpPipeline.fit(df).transform(df)

## 6. Visualize results

In [ ]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("document"),
              F.expr("cols['1']").alias("class")).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|document                                                                                                                                                                                                                                                                  |class   |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Jeu et championnat pas assez excitants ? Devez-vous vérifier l'arbitre vidéo maintenant? Je suis horrifié, pensa-t-il, il ne devrait répondre que s'il a fait des err